# 1. Data Loading and Preliminary Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
file_path = '../data/club_games.csv'
club_games_df = pd.read_csv(file_path)

# Display the first few rows of the dataset for an initial overview
club_games_df.head()


## Initial Observations

Based on the first few rows of the `club_games.csv` dataset, here are some initial observations:

**Structure and Key Variables:**
- `game_id`: Appears to be a unique identifier for each game.
- `club_id`: Identifier for the club.
- `own_goals`: The number of goals scored by the club.
- `own_position`: The club's position in a league or tournament, some values are missing.
- `own_manager_name`: Name of the club's manager.
- `opponent_id`: Identifier for the opponent club.
- `opponent_goals`: Goals scored by the opponent club.
- `opponent_position`: Opponent club's position, also contains missing values.
- `opponent_manager_name`: Name of the opponent club's manager.
- `hosting`: Indicates if the club is hosting the game (Home/Away).
- `is_win`: A binary indicator of whether the club won the game.

**Quality Issues:**
- Missing values are evident in `own_position` and `opponent_position`.

**Data Types:**
Most variables seem correctly typed. However, we'll need to confirm this.


# 2. Data Cleaning and Preprocessing

## 1. Handling Missing Data

We observed missing values in `own_position` and `opponent_position`. We need to decide whether to fill these missing values, drop them, or mark them distinctly.

## 2. Data Type Conversions

We'll confirm the data types of each column and see if any conversions are necessary for more efficient analysis.

## 3. New Features

Assessing the need for creating new features that might aid in our analysis, such as win/loss ratios, average goals per game, etc.

In [ ]:
# Assessing missing values and data types
missing_values = club_games_df.isnull().sum()
data_types = club_games_df.dtypes

# Calculating the percentage of missing values in relation to the entire dataset
total_rows = club_games_df.shape[0]
missing_values_percentage = (missing_values / total_rows) * 100

missing_values_percentage

## Assessment Summary

**Missing Values:**

- `own_position` and `opponent_position` have missing values in 29.83% of the rows. This is a significant portion of the dataset, suggesting that these positions are frequently not recorded or not applicable. We could consider filling these with a placeholder (e.g., -1) or leave them as NaN.

- `own_manager_name` and `opponent_manager_name` are missing in 1.14% of the rows. This is a smaller proportion, but still notable. This could be due to interim periods without a formal manager or data recording issues. We could fill these with 'Unknown' or a similar placeholder.

**Data Types:**

Most columns have appropriate data types.

- `own_position` and `opponent_position` are floats, which could be possible if positions can be non-integer values (e.g., in case of tied positions - although highly unlikely). If not, we might consider converting them to integers or a categorical type. We'll explore the data further to decide in a moment.

**Potential New Features:**

- Win/Loss Ratio: A feature indicating the win/loss ratio of the club could be useful.

- Average Goals per Game: Calculating the average goals scored and conceded per game might offer insights into offensive and defensive strengths.


In [ ]:
# Checking if there are any non-integer values in 'own_position' and 'opponent_position'
non_integer_own_position = club_games_df['own_position'].dropna().apply(float.is_integer).all()
non_integer_opponent_position = club_games_df['opponent_position'].dropna().apply(float.is_integer).all()

non_integer_own_position, non_integer_opponent_position

The analysis reveals that all non-missing values in both `own_position` and `opponent_position` are indeed integers (no floating-point numbers like 2.5, etc.). Therefore, it makes sense to convert these columns to integer data types.



In [ ]:
# Handling missing values
club_games_df['own_position'].fillna(-1, inplace=True)
club_games_df['opponent_position'].fillna(-1, inplace=True)
club_games_df['own_manager_name'].fillna('Unknown', inplace=True)
club_games_df['opponent_manager_name'].fillna('Unknown', inplace=True)

# Converting 'own_position' and 'opponent_position' to integer data types
club_games_df['own_position'] = club_games_df['own_position'].astype(int)
club_games_df['opponent_position'] = club_games_df['opponent_position'].astype(int)

# Verifying the changes
club_games_df.head()

## Actions recap

**Missing Values:**
- Fill missing `own_position` and `opponent_position` with a placeholder (-1).
- Fill missing `own_manager_name` and `opponent_manager_name` with 'Unknown'.

**Data Type Conversions:**
- Convert `own_position` and `opponent_position` to integer data types.

**New Features:**
- Create new features for win/loss ratio and average goals per game.

Next Step: Creating New Features

We proposed adding two new features:

1. Win/Loss Ratio: Calculating the ratio of wins to losses for each club.
2. Average Goals per Game: Determining the average number of goals scored and conceded per game for each club.

Let's proceed with creating these new features. We will first calculate the win/loss ratio for each club. This involves grouping the data by `club_id` and calculating the ratio of wins to the total number of games played by each club. We'll handle cases where the number of losses is zero to avoid division by zero errors.

After that, we'll calculate the average goals per game for each club, both for goals scored and goals conceded.

In [ ]:
# Creating a new feature: Win/Loss Ratio
# Calculating the number of wins and losses for each club
club_wins_losses = club_games_df.groupby('club_id')['is_win'].agg(['sum', 'count'])
club_wins_losses['losses'] = club_wins_losses['count'] - club_wins_losses['sum']

# Avoiding division by zero by adding a small number to losses
club_wins_losses['win_loss_ratio'] = club_wins_losses['sum'] / (club_wins_losses['losses'] + 0.001)

# Merging the win/loss ratio back into the main dataframe
club_games_df = club_games_df.merge(club_wins_losses[['win_loss_ratio']], on='club_id', how='left')

# Creating a new feature: Average Goals per Game
# Calculating average goals scored and conceded per game for each club
average_goals = club_games_df.groupby('club_id').agg({'own_goals': 'mean', 'opponent_goals': 'mean'})
average_goals.rename(columns={'own_goals': 'avg_goals_scored', 'opponent_goals': 'avg_goals_conceded'}, inplace=True)

# Merging the average goals per game back into the main dataframe
club_games_df = club_games_df.merge(average_goals, on='club_id', how='left')

## New Features

- **Win/Loss Ratio (win_loss_ratio):** This column shows the ratio of wins to losses for each club. A small value has been added to losses to avoid division by zero.

- **Average Goals per Game:** Two columns have been added:
    - `avg_goals_scored`: The average number of goals scored by the club per game.
    - `avg_goals_conceded`: The average number of goals conceded by the club per game.


# 3. Exploratory Data Analysis

We will now proceed with the EDA, which includes:

Descriptive Statistics: We'll summarize key columns to understand the distribution and central tendencies of the data.

Visualization: We'll create visualizations to identify patterns and insights, focusing on unique and non-repetitive chart designs.

In-depth Analysis: We'll perform focused analysis on specific areas of interest, such as comparing performance metrics between clubs, analyzing trends over time, etc.

Let's begin with the descriptive statistics to gain an overview of the dataset's key features.

In [ ]:
# Descriptive statistics for key columns
descriptive_stats = club_games_df.describe(include='all')
descriptive_stats.transpose()  # Transposing for better readability

Descriptive Statistics Overview

Here's a summary of key columns in the dataset:

Game and Club Identifiers:
- `game_id`: Ranges from 2,211,607 to 4,228,114.
- `club_id`: Ranges widely from 1 to 112,755.

Goals and Positions:
- `own_goals` and `opponent_goals`: Both have a similar distribution with a mean of approximately 1.46 goals per game and a maximum of 19 goals in a game.
- `own_position` and `opponent_position`: Ranges from -1 (placeholder for missing values) to 21, with an average position around 6.27.

Manager Names:
- `own_manager_name` and `opponent_manager_name`: Feature a large number of unique values (5,578), including 'Unknown' for missing entries.

Hosting:
- `hosting`: Indicates whether the club was hosting the game, with two unique values ('Home' or not).

Win/Loss and Goals:
- `is_win`: Indicates the win status, with about 39% of games resulting in a win for the club.
- `win_loss_ratio`: Ranges significantly, with some extreme values (max is 2000).
- `avg_goals_scored` and `avg_goals_conceded`: Average goals per game are both around 1.46, indicating a balanced offensive and defensive play across clubs.


## Visualizations

We'll be focusing on the following visualizations to start:

1. Distribution of Goals Scored and Conceded: To understand the offensive and defensive capabilities of clubs. We are expecting these to be mirrored distributions, with a similar mean and spread.

2. Win/Loss Ratios Across Clubs: To see how clubs compare in terms of their performance.

3. Position in League vs Performance: Analyzing if there's a correlation between a club's position in the league and their win/loss ratio or goals scored/conceded.


In [ ]:
# Setting the aesthetic style of the plots
sns.set(style="whitegrid")

# Distribution of Goals Scored and Conceded
plt.figure(figsize=(14, 6))

# Goals Scored
plt.subplot(1, 2, 1)
sns.histplot(club_games_df['own_goals'], kde=True, color="skyblue", binwidth=1)
plt.title('Distribution of Goals Scored by Clubs')
plt.xlabel('Goals Scored')
plt.ylabel('Frequency')

# Goals Conceded
plt.subplot(1, 2, 2)
sns.histplot(club_games_df['opponent_goals'], kde=True, color="salmon", binwidth=1)
plt.title('Distribution of Goals Conceded by Clubs')
plt.xlabel('Goals Conceded')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


## Visualization 1: Distribution of Goals Scored and Conceded

The two histograms depict the distribution of goals scored by clubs (in sky blue) and goals conceded (in salmon). 

**Key observations**:

- Symmetry in Scoring and Conceding: Both distributions appear quite similar, reflecting a balance between offensive and defensive capabilities across clubs.

- Common Range: Most games feature between 0 to 3 goals scored or conceded, with a steep drop-off beyond this range.

- Outliers: There are a few instances of exceptionally high goal counts (up to 19), which are rare and could represent extraordinary games.


In [ ]:
# Visualization of Win/Loss Ratios Across Clubs
plt.figure(figsize=(10, 6))
sns.boxplot(x=club_games_df['win_loss_ratio'], color="lightgreen")
plt.title('Win/Loss Ratios Across Clubs')
plt.xlabel('Win/Loss Ratio')
plt.xlim(0, 5)  # Limiting x-axis for better visualization, as there are extreme values
plt.show()

## Visualization 2: Win/Loss Ratios Across Clubs

This boxplot shows the distribution of win/loss ratios across different clubs:

- Central Tendency and Spread: The majority of clubs have a win/loss ratio close to 0.5 to 1, suggesting a balance between wins and losses.

- Outliers: There are clubs with exceptionally high ratios, extending far beyond the main distribution. These could represent clubs with dominant performance or those with very few losses.

- Skewness: The distribution is slightly skewed towards higher ratios, indicating a tendency for more wins than losses across clubs, but this might be influenced by outliers.


In [ ]:
# Visualization: Position in League vs Performance Metrics
plt.figure(figsize=(14, 6))

# Win/Loss Ratio vs Position
plt.subplot(1, 2, 1)
sns.scatterplot(data=club_games_df, x='own_position', y='win_loss_ratio', color="purple", alpha=0.5)
plt.title('Win/Loss Ratio vs League Position')
plt.xlabel('League Position')
plt.ylabel('Win/Loss Ratio')
plt.ylim(0, 5)  # Limiting y-axis for clarity

# Average Goals Scored vs Position
plt.subplot(1, 2, 2)
sns.scatterplot(data=club_games_df, x='own_position', y='avg_goals_scored', color="orange", alpha=0.5)
plt.title('Average Goals Scored vs League Position')
plt.xlabel('League Position')
plt.ylabel('Average Goals Scored')

plt.tight_layout()
plt.show()


## Visualization 3: Position in League vs Performance

**Win/Loss Ratio vs League Position:**
There's a general trend where clubs with lower league positions (towards 1, indicating top positions) tend to have higher win/loss ratios.
Clubs with higher positions (closer to 20) generally show lower ratios, suggesting weaker performance.

**Average Goals Scored vs League Position:**
A similar trend is observed with average goals scored. Top-positioned clubs tend to score more goals on average.
As the league position increases, the average goals scored tend to decrease.

These trends suggest a correlation between a club's league position and their performance in terms of win/loss ratio and goal scoring. Higher-ranked clubs in the league generally perform better.

## Outlier Analysis

To investigate whether the outliers in our data are mistakes or genuine occurrences, we can take the following approach:

1. Statistical Analysis of Outliers: We'll look at the data points that fall beyond a typical range (e.g., beyond 3 standard deviations from the mean) and evaluate their characteristics.

2. Contextual Examination: For data points identified as outliers, we'll examine their context, such as the clubs and managers involved, game IDs, and dates if available. This can help us determine if these outliers correspond to extraordinary but genuine events (like a particularly high-scoring game) or if they seem like data entry errors.

3. Consistency Check: We'll check for consistency in the data. For example, if a club has an extremely high win/loss ratio, we'll see if this is supported by their goals scored and conceded. Inconsistencies might suggest data errors.


In [ ]:
# Function to identify outliers based on standard deviation
def identify_outliers(data, n_std=3):
    mean_val = np.mean(data)
    std_val = np.std(data)
    threshold = n_std * std_val
    outliers = data[np.abs(data - mean_val) > threshold]
    return outliers

# Identifying outliers in win/loss ratio and goal counts
outliers_win_loss_ratio = identify_outliers(club_games_df['win_loss_ratio'])
outliers_goals_scored = identify_outliers(club_games_df['own_goals'])
outliers_goals_conceded = identify_outliers(club_games_df['opponent_goals'])

# Summarizing the outliers
outlier_summary = {
    "Win/Loss Ratio Outliers": outliers_win_loss_ratio.describe(),
    "Goals Scored Outliers": outliers_goals_scored.describe(),
    "Goals Conceded Outliers": outliers_goals_conceded.describe()
}

print(outlier_summary)


**Win/Loss Ratio Outliers:**
- Count: 13 outliers.
- Range: The win/loss ratios for these outliers are extremely high, ranging from 1000 to 2000.
- Implication: Such high ratios are highly unusual and suggest a potential data error or an extraordinary situation. These could be cases where a club has very few losses or data entry errors.

**Goals Scored Outliers:**
- Count: 1,839 outliers.
- Range: Goals scored range from 6 to 19 in these outlier games.
- Implication: While high, these values are within the realm of possibility for high-scoring games. However, the upper end (close to 19 goals) is extremely rare and might warrant further investigation.

**Goals Conceded Outliers:**
- Count: 1,839 outliers, similar to goals scored.
- Range: Goals conceded also range from 6 to 19.
- Implication: This mirrors the goals scored data, indicating that these outliers could represent unusually high-scoring games.



Giving it the benefit of the doubt we'll assume that the goals scored and conceded outliers are genuine and not data entry errors.

## Updated Analysis and Insights

### Revised Win/Loss Ratio Outliers:

Following a deeper investigation into the win/loss ratio outliers, we discovered that these extreme values were not due to sustained exceptional performance or data errors. Instead, they resulted from a methodological issue in our calculation:

**Nature of Outliers:** The clubs with win/loss ratios of 1000 to 2000 had played only a very few games (1 or 2) and won all of them. This led to artificially high ratios due to the lack of losses.

**Adjustment in Approach:** To address this, we've decided to adjust our method for calculating the win/loss ratio. We will either set a minimum number of games threshold for clubs to be included in this calculation or cap the win/loss ratio at a certain value to prevent extreme ratios.

### Implementation of New Method:

We will now recalculate the win/loss ratios using the revised method. This involves either excluding clubs with fewer games or capping the ratio to avoid extreme values.

### Visualization and Re-analysis:

After implementing the new calculation method, we will recreate the visualizations related to the win/loss ratio. This will provide us with an updated and more accurate representation of club performances, free from the distortions caused by clubs with very few games.

### Next Steps:

Once the recalculated win/loss ratios are visualized, we will analyze these results to gain a clearer understanding of club performances across the dataset. This will allow us to draw more reliable conclusions and provide insights that are rooted in a more robust analytical approach.

Let's proceed with applying the new method for calculating win/loss ratios, updating the visualizations, and then analyzing the results. This will form the basis of our final insights and conclusions for the storybook.

In [ ]:
# Implementing the new method for calculating win/loss ratios
# Excluding clubs with fewer than a certain number of games (e.g., 5 games)
min_games_threshold = 5

# Recalculating the win/loss ratio
club_games_grouped = club_games_df.groupby('club_id').agg({'is_win': ['sum', 'count']})
club_games_grouped.columns = club_games_grouped.columns.droplevel(0)
club_games_grouped.rename(columns={'sum': 'wins', 'count': 'games_played'}, inplace=True)

# Filtering clubs that meet the minimum games threshold
filtered_clubs = club_games_grouped[club_games_grouped['games_played'] >= min_games_threshold].copy()

# Calculating revised win/loss ratio
filtered_clubs['losses'] = filtered_clubs['games_played'] - filtered_clubs['wins']
filtered_clubs['revised_win_loss_ratio'] = filtered_clubs['wins'] / (filtered_clubs['losses'] + 0.001)

# Merge revised ratios back into the main DataFrame
club_games_df = club_games_df.merge(filtered_clubs[['revised_win_loss_ratio']], on='club_id', how='left')

# Replace the original win/loss ratio column with the revised one
club_games_df['win_loss_ratio'] = club_games_df['revised_win_loss_ratio']
club_games_df.drop(columns=['revised_win_loss_ratio'], inplace=True)

# Visualization: Revised Win/Loss Ratio
plt.figure(figsize=(10, 6))
sns.histplot(club_games_df['win_loss_ratio'], kde=True, color="lightblue", binwidth=0.1)
plt.title('Revised Win/Loss Ratios Across Clubs')
plt.xlabel('Win/Loss Ratio')
plt.ylabel('Frequency')
plt.xlim(0, 10)  # Limiting x-axis for better visualization
plt.show()

revised_win_loss_ratio_description = club_games_df['win_loss_ratio'].describe()
revised_win_loss_ratio_description

## Visualization Overview:

The histogram displays the distribution of the revised win/loss ratios across clubs. The distribution is more centered and compact, indicating a more realistic representation of club performances.

## Revised Win/Loss Ratio Description:

- Count: The analysis includes 1,483 clubs, each having played at least 5 games.
- Mean Ratio: Approximately 0.68, suggesting a balanced performance across clubs.
- Standard Deviation: About 0.47, indicating some variability in performance but within a reasonable range.
- Range: The ratios range from 0 (no wins) to a maximum of around 4, a significant reduction from the previously observed extreme values.

# 4. Insights and Conclusions

### Key Findings:

**Revised Win/Loss Ratios:**
Implementing a new calculation method for win/loss ratios, which excludes clubs with fewer than five games, yielded more realistic and balanced performance metrics.
The extreme win/loss ratio outliers previously identified were found to be artifacts of clubs with very few games, all of which were won. Adjusting the calculation method mitigated these distortions.

**Goals Scored and Conceded:**
The dataset showed a substantial number of outliers in both goals scored and conceded, ranging from 6 to 19 goals in a game. These outliers were considered plausible within the context of high-scoring games.

**Balanced Offensive and Defensive Capabilities:**
The dataset generally exhibited a balance between offensive (goals scored) and defensive (goals conceded) capabilities across clubs.

### Limitations:

**Data Completeness:**
The dataset had missing values in key columns like 'own_position' and 'opponent_position', which were addressed by filling them with placeholders. 

**Small Sample Size for Certain Clubs:**
Some clubs had a very limited number of games, impacting the reliability of performance metrics like win/loss ratios for these clubs.

### Recommendations:

**Contextual Analysis:**
Future analyses should consider the context of the total number of games played when evaluating performance metrics, especially for clubs with a limited number of games.

**Further Data Verification:**
For extreme outliers, especially in win/loss ratios, further verification with external data sources is recommended to confirm their authenticity.

**Continuous Data Quality Monitoring:**
Regular checks and cleaning routines should be implemented to maintain the quality and accuracy of the dataset.


# Saving the cleaned data

In [ ]:
# Saving the cleaned data
cleaned_data_path = '../data/cleaned/club_games_cleaned.csv'
club_games_df.to_csv(cleaned_data_path, index=False)